In [73]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import re

# Etapa 1: nome dos professores

In [74]:
url = 'http://posfisiologia.ib.usp.br/orientadores.html'
html = requests.get(url)

In [75]:
bs = BeautifulSoup(html.text)

In [76]:
fisio_profs = [x.text for x in bs.find('table',).findAll('span', style ='font-size: 14pt;')]
fisio_profs

['Ana Carolina Inhasz Kiss',
 'Ana Maria de Lauro Castrucci',
 'André Frazão Helene',
 'Cristiéle da Silva Ribeiro',
 'Carlos Arturo Navas Iannini',
 'Débora Rejane Fior Chadi',
 'Fernando Ribeiro Gomes',
 'Gilberto Fernando Xavier',
 'Gisele Akemi',
 'Oda',
 'Gustavo Rohenkohl',
 'José Eduardo P.W. Bicudo',
 'José Guilherme de S. Chaui Berlinck',
 'Lucile Maria Floeter-Winter',
 'Márcio Reis Custódio',
 'Maria Aparecida Visconti',
 'Pedro Augusto C.M. Fernandes',
 'Regina Pekelmann Markus',
 'Renata Guimarães Moreira Whitton',
 'Sandra Márcia Muxel',
 'Silvia Cristina Ribeiro de Souza',
 'Zulma Felisbina da Silva Ferreira']

In [77]:
fisio_profs_reverso = [(nome.split()[-1] + ', ' +' '.join(nome.split()[:-1])) for nome in fisio_profs]
fisio_profs_reverso

['Kiss, Ana Carolina Inhasz',
 'Castrucci, Ana Maria de Lauro',
 'Helene, André Frazão',
 'Ribeiro, Cristiéle da Silva',
 'Iannini, Carlos Arturo Navas',
 'Chadi, Débora Rejane Fior',
 'Gomes, Fernando Ribeiro',
 'Xavier, Gilberto Fernando',
 'Akemi, Gisele',
 'Oda, ',
 'Rohenkohl, Gustavo',
 'Bicudo, José Eduardo P.W.',
 'Berlinck, José Guilherme de S. Chaui',
 'Floeter-Winter, Lucile Maria',
 'Custódio, Márcio Reis',
 'Visconti, Maria Aparecida',
 'Fernandes, Pedro Augusto C.M.',
 'Markus, Regina Pekelmann',
 'Whitton, Renata Guimarães Moreira',
 'Muxel, Sandra Márcia',
 'Souza, Silvia Cristina Ribeiro de',
 'Ferreira, Zulma Felisbina da Silva']


# Etapa 2 - pubmed - busca de autores

In [78]:
dick_pubmed = {}
for prof in fisio_profs_reverso:
    headers_Get = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }


    url_pubmed = f'https://www.ncbi.nlm.nih.gov/pubmed/?term={urllib.parse.quote(prof)}[Full Author Name]&cmd=DetailsSearch'


    html_pubmed = requests.get(url_pubmed,  headers_Get)
    bs_pubmed = BeautifulSoup(html_pubmed.text)

    cod_pubmed = []
    titulo = []
    journal = []
    journal_abbr = []
    autores = []
    year = []

    for x in bs_pubmed.find('div', {'class':'content'}).findAll('a', {'href':re.compile('pubmed/\d')}):
        if 'Free Article' not in x.text :
            if 'Free PMC Article' not in x.text :
                cod_pubmed.append(x['href'].split('/')[2])
                titulo.append(x.text)

    for x in bs_pubmed.find('div', {'class':'content'}).findAll('span', {'class':'jrnl'}):
        journal.append(x['title'])

    for x in bs_pubmed.find('div', {'class':'content'}).findAll('div', {'class':'supp'}):
        autores.append(x.text.split('.')[0])
        journal_abbr.append(x.text.split('.')[1])
        year.append(x.text.split('.')[2].split()[0])
    dick_pubmed[prof] = [autores,titulo,journal,journal_abbr,year,cod_pubmed]


In [79]:
print('\n')

In [80]:
url_pubmed

'https://www.ncbi.nlm.nih.gov/pubmed/?term=Ferreira%2C%20Zulma%20Felisbina%20da%20Silva[Full Author Name]&cmd=DetailsSearch'

In [81]:
import pandas as pd

#pd.DataFrame.from_dict(dick_pubmed)
l = []
for x in list(dick_pubmed):
    df = pd.DataFrame(dick_pubmed[x]).T
    df.columns = 'autores,titulo,journal,journal_abbr,year,cod_pubmed'.split(',')
    df['professor'] = x
    l.append(df)
df_pubmed = pd.concat(l)[['professor','autores','titulo','journal','journal_abbr','year','cod_pubmed']]

In [82]:
df_pubmed[df_pubmed.autores.isna()]

,professor,autores,titulo,journal,journal_abbr,year,cod_pubmed
20,"Floeter-Winter, Lucile Maria",None,"The RelA/cRel nuclear factor-κB (NF-κB) dimer,...",None,None,None,26887983
20,"Muxel, Sandra Márcia",None,The spleen CD4+ T cell response to blood-stage...,None,None,None,21814579


In [83]:
df_pubmed[df_pubmed.professor == 'Muxel, Sandra Márcia']

,professor,autores,titulo,journal,journal_abbr,year,cod_pubmed
0,"Muxel, Sandra Márcia","Aoki JI, Muxel SM, Zampieri RA, Müller KE, Ner...",Author Correction: Differential immune respons...,Scientific reports,Sci Rep,2020,32127634
1,"Muxel, Sandra Márcia","Acuña SM, Floeter-Winter LM, Muxel SM",MicroRNAs: Biological Regulators in Pathogen-H...,Cells,Cells,2020,31906500
2,"Muxel, Sandra Márcia","Aoki JI, Muxel SM, Zampieri RA, Müller KE, Ner...",Differential immune response modulation in ear...,Scientific reports,Sci Rep,2019,31882833
3,"Muxel, Sandra Márcia","Muxel SM, Mamani-Huanca M, Aoki JI, Zampieri R...",Sci Rep. 2020 Mar 4;10(1):4365,International journal of molecular sciences,Int J Mol Sci,2019,32127634
4,"Muxel, Sandra Márcia","Barbosa Lima LE, Muxel SM, Kinker GS, Carvalho...",Metabolomic Profile of BALB/c Macrophages Infe...,Journal of pineal research,J Pineal Res,2019,31835767
5,"Muxel, Sandra Márcia","Aoki JI, Laranjeira-Silva MF, Muxel SM, Floete...",STAT1-NFκB crosstalk triggered by interferon g...,Current opinion in microbiology,Curr Opin Microbiol,2019,31356684
6,"Muxel, Sandra Márcia","Fernandes JCR, Aoki JI, Maia Acuña S, Zampieri...",The impact of arginase activity on virulence f...,Frontiers in cellular and infection microbiology,Front Cell Infect Microbiol,2019,31306995
7,"Muxel, Sandra Márcia","Muxel SM, Acuña SM, Aoki JI, Zampieri RA, Floe...",Melatonin and Leishmania amazonensis Infection...,Frontiers in immunology,Front Immunol,2018,30949455
8,"Muxel, Sandra Márcia","Müller KE, Zampieri RA, Aoki JI, Muxel SM, Ner...",Toll-Like Receptor and miRNA-let-7e Expression...,Parasites & vectors,Parasit Vectors,2018,30555476
9,"Muxel, Sandra Márcia","Acuña SM, Aoki JI, Laranjeira-Silva MF, Zampie...",Amino acid permease 3 (aap3) coding sequence a...,PloS one,PLoS One,2017,30012182


In [84]:
urllib.parse.unquote("https://www.ncbi.nlm.nih.gov/pubmed/?term=Markus%2C%20Regina%5BFull%20Author%20Name%5D&cmd=DetailsSearch")

'https://www.ncbi.nlm.nih.gov/pubmed/?term=Markus, Regina[Full Author Name]&cmd=DetailsSearch'

In [85]:
urllib.parse.quote('https://www.ncbi.nlm.nih.gov/pubmed/?term=Markus, Regina[Full Author Name]&cmd=DetailsSearch')

'https%3A//www.ncbi.nlm.nih.gov/pubmed/%3Fterm%3DMarkus%2C%20Regina%5BFull%20Author%20Name%5D%26cmd%3DDetailsSearch'

# Etapa 2: link do google scholar dos professores

In [86]:
fisio_profs_google_scholar = ["+".join([x +' google scholar'][0].split()) for x in fisio_profs]

fisio_profs_google_scholar

['Ana+Carolina+Inhasz+Kiss+google+scholar',
 'Ana+Maria+de+Lauro+Castrucci+google+scholar',
 'André+Frazão+Helene+google+scholar',
 'Cristiéle+da+Silva+Ribeiro+google+scholar',
 'Carlos+Arturo+Navas+Iannini+google+scholar',
 'Débora+Rejane+Fior+Chadi+google+scholar',
 'Fernando+Ribeiro+Gomes+google+scholar',
 'Gilberto+Fernando+Xavier+google+scholar',
 'Gisele+Akemi+google+scholar',
 'Oda+google+scholar',
 'Gustavo+Rohenkohl+google+scholar',
 'José+Eduardo+P.W.+Bicudo+google+scholar',
 'José+Guilherme+de+S.+Chaui+Berlinck+google+scholar',
 'Lucile+Maria+Floeter-Winter+google+scholar',
 'Márcio+Reis+Custódio+google+scholar',
 'Maria+Aparecida+Visconti+google+scholar',
 'Pedro+Augusto+C.M.+Fernandes+google+scholar',
 'Regina+Pekelmann+Markus+google+scholar',
 'Renata+Guimarães+Moreira+Whitton+google+scholar',
 'Sandra+Márcia+Muxel+google+scholar',
 'Silvia+Cristina+Ribeiro+de+Souza+google+scholar',
 'Zulma+Felisbina+da+Silva+Ferreira+google+scholar']

# Etapa 3: acessar a página com publicações de cada professor

In [87]:
print(f'https://www.google.com/#q={fisio_profs_google_scholar[2]}')

https://www.google.com/#q=André+Frazão+Helene+google+scholar


In [88]:
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }

dick = {}
for prof in fisio_profs_google_scholar:
    r = requests.get(f'https://www.google.com/search?q={"+".join(prof.split())}', headers_Get)

    bs2 = BeautifulSoup(r.text)
    try:
        x = bs2.findAll('a', href = re.compile('scholar.google'))[0]['href'][7:]

        x2 = urllib.parse.unquote(x)

        x3 = requests.get(x2, headers_Get)

        x4 = BeautifulSoup(x3.content)

        artigos = []
        citacoes = []
        autores = []
        ano = []

        for link in x4.findAll('td', {"class":"gsc_a_t"}):
            artigos.append(link.find('a').text)
            autores.append(link.find('div',{'class':'gs_gray'}).text)
        for link in x4.findAll('td', {'class': 'gsc_a_c'}):
            citacoes.append(int(link.find('a').text))
        nome_prof = '_'.join(x4.find('div', id = 'gsc_prf_in').text.split())
        for link in x4.findAll('td', {"class":"gsc_a_y"}):
            ano.append(int(link.find('span').text))

        dick[nome_prof] = list(zip(artigos,citacoes,autores,ano))
    except IndexError: pass

list(dick)
                     

['Ana_Carolina_Kiss',
 'Ana_Maria_de_Lauro_Castrucci',
 'André_Frazão_Helene',
 'Cristiéle_da_Silva_Ribeiro',
 'Carlos_Arturo_Navas',
 'Debora_Rejane_Fior_Chadi',
 'Fernando_Ribeiro_Gomes',
 'Gilberto_Fernando_Xavier',
 'Gisele_Akemi_Oda',
 'Tomohiro_Oda',
 'Gustavo_Rohenkohl',
 'Jose_R_Bicudo',
 'José_Guilherme_Chaui-Berlinck',
 'Lucile_Maria_Floeter_Winter',
 'Marcio_Reis_Custodio',
 'Maria_Aparecida_Visconti',
 'Pedro_Augusto_Carlos_Magno_Fernandes',
 'Hazel_Markus',
 'Renata_Guimarães_Moreira',
 'Cristina_Ribeiro',
 'Zulma_Ferreira']

In [89]:
import pandas as pd

In [90]:
l = []
for x in list(dick):
    
    df = pd.DataFrame(dick[x], columns = ['artigo','citacoes','autores','ano'])
    df['professor'] = x
    l.append(df)
df_profs = pd.concat(l)[['professor','artigo','autores','ano','citacoes']]
df_profs.autores.values

array(['ACI Kiss, PHO Lima, YK Sinzato, M Takaku, MA Takeno, MVC Rudge, ...',
       'YK Sinzato, PHO Lima, KE Campos, ACI Kiss, MVC Rudge, ...',
       'DC Damasceno, ACI Kiss, YK Sinzato, KE De Campos, MVC Rudge, ...',
       'ACI Kiss, JE de Carvalho, CA Navas, FR Gomes',
       'DC Damasceno, GT Volpato, YK Sinzato, PHO Lima, MSS Souza, IL Iessi, ...',
       'ACI Kiss, B Woodside, LF Felício, J Anselmo-Franci, DC Damasceno',
       'AH dos Santos, ML Vieira, N de Azevedo Camin, JA Anselmo-Franci, ...',
       'A Kiss, M Takaku, DC Damasceno, KE Campos, YK Sinzato, PO Lima, ...',
       'ACI Kiss, B Woodside, YK Sinzato, MM Bernardi, WDG Kempinas, ...',
       'AH dos Santos, AC Ramos, KM Silveira, ACI Kiss, R Longhini, A Diniz, ...',
       'N de Azevedo Camin, ML Vieira, BG Montagnini, ACI Kiss, DCC Gerardin',
       'ML Vieira, AH dos Santos, LS Silva, GSA Fernandes, ACI Kiss, ...',
       'MO Klein, H MacKay, A Edwards, SB Park, ACI Kiss, LF Felicio, A Abizaid',
       'AC Ramo

In [91]:
df_profs.groupby('professor').ano.agg(['sum','mean','min','max']).sort_values('sum', ascending = False)

,sum,mean,min,max
professor,,,,
Tomohiro_Oda,40275,2013.75,2011,2018
Ana_Carolina_Kiss,40258,2012.90,2006,2019
Pedro_Augusto_Carlos_Magno_Fernandes,40256,2012.80,2005,2018
Gustavo_Rohenkohl,40253,2012.65,2006,2018
Renata_Guimarães_Moreira,40230,2011.50,2005,2018
Cristiéle_da_Silva_Ribeiro,40226,2011.30,2001,2018
André_Frazão_Helene,40194,2009.70,2003,2018
Cristina_Ribeiro,40174,2008.70,1999,2017
Gisele_Akemi_Oda,40173,2008.65,1995,2017
